In [ ]:
## Goal: Prep product descriptions to data to be clustered by a BERT algo
# Q: What does this file do? 
# A: 
# FROM lines like this:

# TO lines like this: 
#A|156|1|Key Moments Keychain|LUxAIVS|On the Move|-brand love!clip ring allows you to easily switch around your keys fabric is water resistant and easy to wipe clean

# Q: What are the diff columns? 
# First = letter id pointing to a description, which the UI will use later on
# Second = char count of the description because curios if that will impact BERT algo
# Third = near dup count - Not sure what to do with this one - maybe nothing
# Fourth = one-way-scrabled product id
# Fifth = activity ( i.e., LOB) 
# Sixth = the actual description 




In [1]:
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
import torch
import re
import pandas as pd
from datetime import datetime

def getRightNow():
    now = datetime.now()
    hms = now.strftime("%H:%M:%S")
    return hms
print("Imports {}".format(getRightNow()))

Imports 11:00:27


In [18]:
def ETL(outfile, infile):
    
    def numberToLetter(num):
        if num <= 0:
            raise ValueError("Input must be a positive integer.")    
        letters = ""
        while num > 0:
            num -= 1
            # Map the remainder to the corresponding letter
            letters = chr(num % 26 + ord('A')) + letters  
            # Divide by 26 to move to the next digit
            num //= 26
        return letters
    
    def makePrettyRow(x,  dupCount, ith):         
        lookUpId = numberToLetter(ith)
        theImportantStuff = "{}|{}|{}|{}".format(x["name"], x["id"],x["activity"],x["desc"])
        n = len(theImportantStuff)
        # lookUpId = row look up...  so 'XYZ' will point to 'some description'
        # n = char count --- curious how length affects the BERT model
        # dupCount --- might want to weight things somewhat depending on how duplicated they are ( assumption = more dups is more good )
        return "{}|{}|{}|{}\n".format(lookUpId, n, dupCount, theImportantStuff)
    
    def nothingIsEmpty(ary): 
        # Lots of the cells are empty and illformed - skip any row with an empty cell
        isOk = len(ary[0]) > 0 and  len(ary[1]) > 0 and len(ary[2]) > 0 and len(ary[3]) > 20 
        return isOk 
    
    def convert_numbers_to_x(string):
        # Convert "blah blah size 6" or "blah blah size 16" to "blah blah size x"
        # Needed to squish near duplicated lines
        return re.sub(r'\d+', 'x', string)

    descriptions = [] 
    seen = {} 
    products = [] 
    with open(infile, 'r') as file:
        for line in file:
            pieces = line.replace('"', '').strip().split('|')
            if len(pieces) == 4:
                pieces[3] = convert_numbers_to_x(pieces[3])
                if nothingIsEmpty(pieces) == True: 
                    x = {
                        "name":pieces[0],
                        "id":pieces[1],
                        "activity":pieces[2],
                        "desc":pieces[3]
                    }
                    if x['desc'] not in seen: 
                        seen[x["desc"]] = 1
                        products.append(x)
                    else:
                        seen[x["desc"]] += 1
                        
                        
    with open(outfile, 'w') as file:
        i = 0
        # for x in products[0:3]:
        for x in products:
             i += 1
             r = makePrettyRow(x,  seen[x["desc"]], i)
             file.write(r)
    print("ETL and wrote to {} with {} rowcount at {}".format(outfile, len(products), getRightNow()))            
ETL("ready_for_analysis.txt", "Data_June_16_redacted.txt")

ETL and wrote to ready_for_analysis.txt with 2511 rowcount at 11:18:08
